The Basic OpenAI Chat Completion approach using the openai SDK (specifically with AzureOpenAI) is ideal for quick prototyping with Azure-hosted OpenAI models like GPT-3.5 or GPT-4. It uses a familiar API format similar to OpenAI's public APIs, where prompts are passed in a plain chat structure. This method is well-suited for developers who want to get started quickly and build simple applications using text-only prompts. It allows optional authentication via API key and is primarily focused on OpenAI-compatible chat models deployed in the Azure OpenAI resource.

In contrast, the AI Foundry model integration via the azure-ai-inference SDK is a more enterprise-focused approach. It supports a wider range of models—including non-OpenAI LLMs like LLaMA, Mistral, and Meta models—that are hosted through Azure AI Studio or AI Foundry. This SDK uses a more structured format for inputs and supports multimodal capabilities, such as combining text and images. It requires AzureKeyCredential for stricter security and is optimized for integration with production environments like Azure ML, Databricks, and Mosaic AI. While it's slightly more verbose to implement, it is more powerful and flexible for large-scale or diverse use cases.

##Install the openai SDK

In [0]:
%pip install openai==1.56.0

##Restarting our Python Environment

In [0]:
dbutils.library.restartPython()

##Creating an Azure OpenAI Client

In [0]:
from openai import AzureOpenAI

openai_client = AzureOpenAI(
    azure_endpoint = "YOUR-AZURE-OPENAI-ENDPOINT",
    api_key = "YOUR-AZURE-OPENAI-API-KEY",
    api_version="2024-02-15-preview"
)
     

#Writing code for the Chat Completions API
#Defining the system prompt

In [0]:
system_prompt = f""" You are a helpful AI assistant meant to answer the user query """

##Defining the user prompt/ user query

In [0]:

user_prompt = f""" Hi how are you?"""

##Making a call to the GPT engine

In [0]:
gpt_response = openai_client.chat.completions.create(
    model = "YOUR-MODEL-NAME",
    messages = [
        {
            "role":"system", "content": f"{system_prompt}"
        },
        {
            "role":"user", "content": f"{user_prompt}"
        }
    ],
    temperature = 0.7
)

print(gpt_response.choices[0].message.content)

#Generating a custom Python function using Mlflow that can be served as a Mosaic AI endpoint

In [0]:
import mlflow
from mlflow import pyfunc
from openai import AzureOpenAI

class BasicChatBot(pyfunc.PythonModel):
      def __init__(self, GPT_model):
          self.GPT_model = GPT_model

      def chatCompletionsAPI(self, user_query):
          
          openai_client = AzureOpenAI(
            azure_endpoint = "YOUR-AZURE-OPENAI-ENDPOINT",
            api_key = "YOUR-AZURE-OPENAI-API-KEY",
            api_version = "2024-02-15-preview"
          )
          response = openai_client.chat.completions.create(
            model = self.GPT_model,
            messages = [
                {
                    "role":"system",
                    "content":"You are a helpful AI assistant"
                },
                {
                    "role":"user",
                    "content":f"{user_query}"
                }
            ],
            temperature = 0.7
          )

          return response.choices[0].message.content

      def predict(self, context, data):
          user_query = data["user_query"].iloc[0]
          gpt_response = self.chatCompletionsAPI(user_query)
          return gpt_response

      

##Saving our model

In [0]:
test_model = BasicChatBot(GPT_model="YOUR_MODEL_NAME")

In [0]:

from mlflow.models import infer_signature
import pandas as pd

signature = infer_signature(pd.DataFrame([{"user_query": "hello how are you?"}]))
model_path = "basicchatbot"
mlflow.pyfunc.save_model(path=model_path, python_model=test_model, signature=signature)

##Loading our Saved Model

In [0]:

# Load our custom model from the local artifact store
loaded_pyfunc_model = mlflow.pyfunc.load_model(model_path)


##Testing our saved model

In [0]:

model_input = pd.DataFrame([{"user_query": "hello how are you?"}])

model_response = loaded_pyfunc_model.predict(model_input)

print(model_response)

##Logging our saved/loaded model

In [0]:

import mlflow

# Log the model as an artifact
with mlflow.start_run() as run:
    mlflow.log_artifacts(local_dir=model_path, artifact_path="BasicChatBot")
    print(f"Model logged with run ID: {run.info.run_id}")

##Inferencing the real-time endpoint

In [0]:

{
  "dataframe_records":[
    {
        "user_query":"tell me something about India"
    }
  ]
}